In [1]:
# init
from os import path
import numpy as np
from pynput import keyboard
from pedalboard import Pedalboard, Chorus, Reverb
from pedalboard.io import AudioFile
import music21
from keyboard_layout import KeyboardLayout
from tone import Tone
from audio_player import AudioPlayer
from time_going import Timeline

smplRate = 44100

board = Pedalboard([Chorus(), Reverb(room_size=0.25)])

keyboard_layout = KeyboardLayout()
keyboard_layout.load('complete left to right')


complete left to right loaded as keyboard layout successfully.


In [43]:

harmonics = [(np.power(2.1,np.log2(i)),1) for i in range(2,10)]
print(harmonics)
tonemaker_1 = Tone(harmonics = harmonics, fade_duration = 0.06)

def scaling_1(point):
    return 40 * np.power(2.1,point/12)

def scaling_2(point):
    scale = [2,2,1,2,2,2,1]
    result = 1
    for i in range(point):
        result += scale[i%len(scale)]
    return result

def final_scaling(x):
    return scaling_1(scaling_2(x))



[(2.1, 1), (3.2411975291572204, 1), (4.41, 1), (5.599767983546668, 1), (6.806514811230163, 1), (8.027566705613907, 1), (9.261000000000001, 1), (10.50536142301487, 1)]


In [3]:
timeline = Timeline(IS_LOOPED = False)

In [4]:
audioplayer = AudioPlayer(timeline,board)
audioplayer.start()

In [44]:
keyboard_layout.start(final_scaling, audioplayer, tonemaker_1, timeline)

started...


This process is not trusted! Input event monitoring will not be possible until it is added to accessibility clients.


stoped.


In [37]:
import time
score = music21.corpus.parse('bach/bwv65.2.xml')
# score.show()
key = score.analyze('key')
tonicPitchClass = key.tonic.pitchClass
scoreChordify = score.chordify()
scoreChordsFlat = scoreChordify.flatten()
chords = scoreChordsFlat.getElementsByClass('Chord')
for chord in chords:
    print([(p.midi-40) for p in chord.pitches],chord.duration.quarterLength)
    position = audioplayer.position

    for p in chord.pitches:
        fundamental = scaling_1(p.midi-20)
        tone = tonemaker_1.make(fundamental)/10
        timeline.add(tone, position)
    time.sleep(chord.duration.quarterLength/2)
    
    # i.key = k


    # rootPitchClass = chord.root().pitchClass
    # diff = rootPitchClass - tonicPitchClass
    # if(diff < 0):
    #     diff += (11+1)
    # print(diff)


# print(s.metadata.all())

[5, 20, 24, 29] 1.0
[17, 20, 24, 29] 1.0
[15, 20, 24, 29] 1.0
[14, 22, 29] 1.0
[15, 22, 27, 31] 1.0
[10, 22, 27, 31] 1.0
[13, 22, 27, 31] 1.0
[12, 20, 27, 32] 1.0
[8, 24, 27, 31] 1.0
[13, 20, 25, 29] 1.0
[8, 20, 24, 27] 3.0
[8, 24, 27, 32] 1.0
[20, 24, 27, 32] 1.0
[19, 24, 28, 34] 1.0
[17, 24, 29, 32] 1.0
[14, 22, 29, 32] 1.0
[15, 22, 27, 31] 1.0
[8, 24, 27, 32] 2.0


KeyboardInterrupt: 

In [11]:
# closing audio in case of emergency
audioplayer.stop()

AttributeError: 'NoneType' object has no attribute 'close'

In [24]:
timeline.clear()

In [5]:
timeline.save()

didn't save.


stoped.


In [5]:
timeline.load()

test1 loaded as timeline successfually.
